In [1]:
import gym # openAi gym
from gym import envs
import numpy as np 
import pandas as pd 
import random

In [2]:
import warnings
warnings.filterwarnings('ignore')


In [3]:
env = gym.make('Taxi-v3')   # Here you set the environment
env._max_episode_steps = 40000
env.reset()

169

In [116]:
def policy_evaluation(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Implement the policy evluation algorithm here given a policy and a complete model of the environment.
    
    
    Arguments:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: This is the minimum threshold for the error in two consecutive iteration of the value function.
        discount_factor: This is the discount factor - Gamma.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
#     Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        delta = 0
        for s in range(env.nS):
            vNew = 0
            for a in range(env.nA):
                for prob, nextState, reward, done in env.P[s][a]:
                    vNew+=policy[s][a] * prob * (reward + discount_factor*V[nextState])
            
            delta = max(delta, np.abs(V[s]-vNew))
            V[s] = vNew
                
#         print(delta)
        if delta < theta:
            break

    print(V)
    
    return np.array(V)


In [97]:
def policy_iteration(env, policy_eval_fn=policy_evaluation, discount_factor=1.0):
    """
    Implement the Policy Improvement Algorithm here which iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Arguments:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    def one_step_lookahead(state, V):
        """
        Implement the function to calculate the value for all actions in a given state.
        
        Arguments:
            state: The state to consider (int)
            V: The value to use as an estimator, Vector of length env.nS
        
        Returns:
            A vector of length env.nA containing the expected value of each action.
        """
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, nextState, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[nextState])

        return A


    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    numIterations = 0

    while True:
        numIterations += 1
        
        V = policy_eval_fn(policy, env, discount_factor)
        policyStable = True
        
        for s in range(env.nS):
            oldAction = np.argmax(policy[s])

            qValues = one_step_lookahead(s, V)
            newAction = np.argmax(qValues)

            if oldAction != newAction:
                policyStable = False
                        
            policy[s] = np.zeros([env.nA])
            policy[s][newAction] = 1

        if policyStable:
            print(numIterations)
            return policy, V
    
    return policy, np.zeros(env.env.nS)

In [98]:
env.reset()
policyPI, valuePI = policy_iteration(env, discount_factor=0.99)

12


In [103]:
print(policyPI)

[[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 ...
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


In [107]:
print(valuePI)

[944.72316569 864.01270478 903.55686813 873.75019718 789.53759087
 864.01270478 789.53757231 816.76645272 864.01272333 826.02673924
 903.55686813 835.38053503 807.59881631 826.02673924 807.59879776
 873.75019718 955.27593403 873.75021631 913.69381566 883.58606752
 934.27593403 854.37257773 893.52129945 864.01269521 798.52282815
 873.75021631 798.52280978 826.02672968 854.3725961  816.76647185
 893.52129945 826.02672968 816.76649022 835.38055415 816.76647185
 883.58606752 944.72317469 883.58608645 903.5568775  893.52128998
 883.58610464 807.59880713 844.82885195 816.76646238 844.82887014
 923.9331565  844.82885195 873.75020684 844.82887014 807.59880713
 883.58608645 816.76646238 826.0267668  844.82885195 826.02674861
 893.52128998 893.52132673 934.27592494 893.52130873 903.55686813
 873.7502436  798.52281906 835.38056343 807.59879776 854.37260501
 934.27592494 854.37258701 883.58607708 835.38058144 798.52281906
 873.75022559 807.59879776 835.38058144 854.37258701 835.38056343
 903.55686

In [111]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    This section is for Value Iteration Algorithm.
    
    Arguments:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: Stop evaluation once value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """
    
    def one_step_lookahead(state, V):
        """
        Function to calculate the value for all actions in a given state.
        
        Arguments:
            state: The state to consider (int)
            V: The value to use as an estimator, Vector of length env.nS
        
        Returns:
            A vector of length env.nA containing the expected value of each action.
        """
        A = np.zeros(env.env.nA)
        for a in range(env.nA):
            for prob, nextState, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[nextState])

        return A
    
    V = np.zeros(env.env.nS)
    
    numIterations = 0
    
    while True:
        numIterations += 1
        delta = 0
        
        for s in range(env.nS):
            qValues = one_step_lookahead(s, V)
            newValue = np.max(qValues)
            
            delta = max(delta, np.abs(newValue - V[s]))
            V[s] = newValue
        
        if delta < theta:
            break
    
    policy = np.zeros([env.nS, env.nA])
    for s in range(env.nS):  #for all states, create deterministic policy
        qValues = one_step_lookahead(s,V)
        
        newAction = np.argmax(qValues)
        policy[s][newAction] = 1
    
    print(numIterations)    
    return policy, V

In [112]:
policyVI,valueVI = value_iteration(env, discount_factor=0.99)

610


In [113]:
print(policyVI)

[[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 ...
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]


In [114]:
print(valueVI)

[944.71905319 864.00842358 903.55258694 873.74578494 789.53347837
 864.00842358 789.53329112 816.76204048 864.00861084 826.02245805
 903.55258694 835.37612279 807.59470382 826.02245805 807.59451656
 873.74578494 955.27186266 873.74597793 913.68957728 883.5816994
 934.27186266 854.36833935 893.51706107 864.00832709 798.51875678
 873.74597793 798.5185714  826.02236156 854.36852473 816.76223347
 893.51706107 826.02236156 816.76241885 835.37631577 816.76223347
 883.5816994  944.71914403 883.58189046 903.5526815  893.51696554
 883.58207398 807.59461113 844.82465595 816.76213794 844.82483948
 923.9289605  844.82465595 873.74588241 844.82483948 807.59461113
 883.58189046 816.76213794 826.02273614 844.82465595 826.02255261
 893.51696554 893.51733638 934.2717709  893.51715469 903.55258694
 873.74625324 798.51866502 835.37640939 807.59451656 854.36861466
 934.2717709  854.36843297 883.58179589 835.37659109 798.51866502
 873.74607155 807.59451656 835.37659109 854.36843297 835.37640939
 903.552586

In [ ]:
def Q_learning_train(env,alpha,gamma,epsilon,episodes): 
    """Q Learning Algorithm with epsilon greedy policy

    Arguments:
        env: Environment 
        alpha: Learning Rate --> Extent to which our Q-values are being updated in every iteration.
        gamma: Discount Rate --> How much importance we want to give to future rewards
        epsilon: Probability of selecting random action instead of the 'optimal' action
        episodes: No. of episodes to train 

    Returns:
        Q-learning Trained policy

    """
    
    """Training the agent"""

    
    #Initialize Q table here
    q_table = np.zeros([env.observation_space.n, env.action_space.n])  
    
    for i in range(1, episodes+1):
        state = env.reset()
        # implement the Q-learning algo here

       # Start with a random policy
    policy = np.ones([env.env.nS, env.env.nA]) / env.env.nA

    for state in range(env.env.nS):  #for each states
        #Extract the best optimal policy found so far
        
    
    return policy, q_table

In [117]:
for x in range(len(policyPI[0])):
    if not (policyPI[0][x] == policyVI[0][x]).all():
        print("Not the same Policy")
        break
print("Same Policy")


Same Policy
